<a href="https://colab.research.google.com/github/airesx2/Internship-2025/blob/main/cudaq_learning%20/cudaq_bruteForce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cudaq

In [1]:
import cudaq

In [9]:
#Password Brute Force with grover's algorithm
#Starting with 2 qubits (hardcoded)

#Had to move this segment out b/c cudaq didn't support >>
def target_to_bits(target, n):
    #>> shifts bits of a number to the right
    return [(target >> (n - 1 - i)) & 1 for i in range(n)]


@cudaq.kernel #remember the decorator!
def search(bit_pattern: list[int]):

  #Step 1: Create state 0 qubits
  n = len(bit_pattern)
  qubits = cudaq.qvector(n) #initialize n qubits

  #Step 2: Superposition
  for q in qubits:
    h(q) #apply H gate to all qubits

  #looping for better success probability
  for i in range(n*2):

    #Step 3: Oracle
    for i in range(n): #apply x gates on qubits that should be 0 in the target state
      if bit_pattern[i] == 0:
        x(qubits[i])

    cz(qubits[0], qubits[1]) #flip target state

    for i in range(n): #apply x gates on qubits that should be 0 in the target state
      if bit_pattern[i] == 0:
        x(qubits[i])

    #Step 4: Diffusion
    #reflects the state through origin
    for q in qubits: #puts qubits in "mirror state"
      h(q) #back to superposition
      x(q) #flip qubit

    #reflection step
    cz(qubits[0], qubits[1]) #state vector reflected around the mean amplitude

    for q in qubits: #undo
      x(q)
      h(q)

  #Step 5: Measure
  for q in qubits:
        mz(q)

#Step 6: Run & Sample

#inputs
n = 2 #number of qubits
target = 3
bit_pattern = target_to_bits(target, n)

#run
result = cudaq.sample(search, bit_pattern) #run sample
target_bin = format(target, f'0{n}b') #convert target to binary
result_dict = {key: result.count(key) for key in result} #loops over bitstrings, gets total count
success_count = result.count(target_bin) #count no. correct state, only accepts binary
total_shots = sum(result_dict.values()) #total number of shots (default is 1000)
success_rate = (success_count / total_shots) * 100 #calculate success rate as %
print(f"Result for target = |{target:02b}⟩:")
print(result)
print(f"Success rate: {success_rate:.2f}%")



Result for target = |01⟩:
{ 01:1000 }

Success rate: 100.00%
